In [ ]:
import cv2
import numpy as np
import os
import random

In [ ]:
# Define augmentation functions
def rotate_image(image, angle):
    center = (image.shape[1] // 2, image.shape[0] // 2)
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    return cv2.warpAffine(image, matrix, (image.shape[1], image.shape[0]), borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))

def shear_image(image, shear_factor):
    rows, cols, ch = image.shape
    matrix = np.float32([[1, shear_factor, 0], [0, 1, 0]])
    return cv2.warpAffine(image, matrix, (cols, rows), borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))

def add_noise(image):
    noise = np.random.normal(0, 10, image.shape)
    noisy_image = np.clip(image + noise, 0, 255).astype(np.uint8)
    return noisy_image

def adjust_brightness_contrast(image, brightness=0, contrast=0):
    beta = brightness  # Simple brightness control
    alpha = contrast  # Simple contrast control
    return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

def augment_image(image):
    # Apply a random small rotation
    rotated_image = rotate_image(image, np.random.uniform(-1, 1))
    
    # Apply random shear
    sheared_image = shear_image(rotated_image, np.random.uniform(-0.1, 0.1))
    
    # Apply random noise
    noisy_image = add_noise(sheared_image)
    
    # Adjust brightness and contrast
    augmented_image = adjust_brightness_contrast(noisy_image, brightness=np.random.uniform(-10, 50), contrast=np.random.uniform(0.9, 1.8))
    
    return augmented_image

def process_folder(folder_path, output_base_dir, num_augmented_images=20):
    output_dir = os.path.join(output_base_dir, os.path.basename(folder_path))
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('png', 'jpg', 'jpeg')):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            
            for i in range(num_augmented_images):
                augmented_image = augment_image(image)
                cv2.imwrite(os.path.join(output_dir, f'{os.path.splitext(filename)[0]}_aug_{i}.png'), augmented_image)

    # Ensure the number of images does not exceed 80
    limit_images(output_dir, 80)

def limit_images(folder_path, max_images):
    current_image_count = len(os.listdir(folder_path))
    if current_image_count > max_images:
        images_to_delete = current_image_count - max_images
        print(f"Folder {folder_path} has {current_image_count} images. Deleting {images_to_delete} extra images.")

        # Get all image filenames
        image_filenames = os.listdir(folder_path)
        
        # Select random images to delete
        images_to_delete_filenames = random.sample(image_filenames, images_to_delete)
        
        # Delete the selected images
        for filename in images_to_delete_filenames:
            os.remove(os.path.join(folder_path, filename))

def process_main_folder(main_folder_path, num_augmented_images=20):
    output_base_dir = 'augmented_images'
    for subfolder in os.listdir(main_folder_path):
        subfolder_path = os.path.join(main_folder_path, subfolder)
        if os.path.isdir(subfolder_path):
            process_folder(subfolder_path, output_base_dir, num_augmented_images)

main_folder_path = 'less_images/dha_up'  # Replace with the path to your main folder where clusters have very less images
process_main_folder(main_folder_path)
